In [ ]:
import folium
from folium.plugins import Draw
import ogr, os
from IPython.display import Image
from shapely.geometry import mapping, shape
from shapely.geometry import box as shBox
import rasterio
import pandas as pd
import geopandas as gpd
import gdal
from shapely.geometry import Polygon
from shapely.geometry import mapping, LineString, MultiLineString
import shapefile

#Needed external file: download US coast outline from: https://geodata.lib.berkeley.edu/catalog/stanford-xv279yj9196

#Footprint data
foot= '.../'
#ROI name
roi="NorthCarolina"
#Set buffer distance in decimal degreees
dist=0.01



#Use Folium to draw bounding box, click export and save once done.
world=folium.Map(location=[36.1487072100852, -75.73136858295845], zoom_start=9, tiles='Stamen Terrain')
draw= Draw(export=True,
           filename='tempbounds.geojson')
draw.add_to(world)
world


In [ ]:
bounds='.../Downloads/tempbounds.geojson'

#Import Line file of US Coast:
US_cst= gpd.read_file('.../usa_coastl.shp')#Downloaded file from https://geodata.lib.berkeley.edu/catalog/stanford-xv279yj9196
US_cst=US_cst.to_crs('EPSG:4326')



bnd=gpd.read_file(bounds)
bnd=bnd.set_crs('EPSG:4326')

#clip coastal polyline
roi_coast=geopandas.clip(US_cst,bnd)
roi_coast=roi_coast.to_crs('EPSG:4326')
buffer=roi_coast.buffer(dist)

print('buffer created')

json=gpd.GeoDataFrame(gpd.read_file(foot))
json_clipped = geopandas.clip(json, buffer)
json_clipped.to_file('.../ROI_Bldgs.geojson', driver='GeoJSON')